In [1]:
import copy
import sys
sys.path.insert(0, "../../")
from classes.atom_classes.atom import Atom
from classes.atom_classes.atom_file_handler import Atom_File_handler
from classes.atom_classes.atom_collection import Atom_Collection
from classes.atom_classes.atom_collection_plotter import Animator, Atom_Collection_Plotter
from classes.calculators.lennardjones import LJGauss
from classes.optimizers.random_struct_search import Line_searcher1
import numpy as np

In [2]:
class Atom_plotter(Animator):
    def __init__(self, default_cols=("C0", "C1"), markersize=50, scale_with_mass=False, alpha=1) -> None:
        super().__init__(default_cols, markersize, scale_with_mass, alpha)

In [3]:
def make_random_conf(N_atoms=12, unit_cell=(np.array([10.0,0.0]), np.array([0.0,10.0])), pbc=True):
    random_poses = np.random.rand(N_atoms, 2)*np.linalg.norm(unit_cell[0])
    atomlist = [Atom(position=pos) for pos in random_poses]
    atom_col = Atom_Collection(atomlist=atomlist, unit_cell=unit_cell, pbc=pbc)
    atom_col.calculator = LJGauss()
    return atom_col


opt_cols = []

for i in range(2000):
    optimizer = Line_searcher1(atom_col=make_random_conf())
    optimizer.run(track=False)
    opt_cols.append(optimizer.best_atom_col)

c:\Users\karst\Computational-Physics\Week7\exercise 38\../..\classes\calculators\lennardjones.py:44: RuntimeWarning: invalid value encountered in multiply
  gauss_term = self.gauss_eps/self.gauss_sigma2*np.exp(-(r-self.r0)**2/(2.0*self.gauss_sigma2))*(r-self.r0)


KeyboardInterrupt: 